# Create Reach Network KG

## Imports

In [1]:
from simpledbf import Dbf5
import pandas as pd
import geopandas as gpd

from rdflib import Graph, Literal, Namespace
from rdflib.namespace import GEO, OWL, PROV, RDF, RDFS, XSD

import json
from shapely import to_geojson

PyTables is not installed. No support for HDF output.
SQLalchemy is not installed. No support for SQL output.


## Import Flowlines and Flow Connections

In [2]:
### Input ###
flowplus_file = r'../Geospatial/NE_01_NHDPlusAttributes/PlusFlow.dbf'
flowline_file = r'../Geospatial/NE_01_NHDSnapshot/NHDFLowline.shp'
# flowline_file = r'../Geospatial/NHDFLowline_BBox.shp'

### Output ###
ttl_file = '../ttl files/me_reaches.ttl'
# ttl_file = '../ttl files/reaches_bbox.ttl'

In [3]:
dbf = Dbf5(flowplus_file)
plusflow = dbf.to_dataframe()
plusflow.drop(['FROMHYDSEQ',
               'FROMLVLPAT', 
               'TOHYDSEQ',
               'TOLVLPAT',
               'NODENUMBER',
               'DELTALEVEL',
               'DIRECTION', 
               'GAPDISTKM',
               'HasGeo',
               'TotDASqKM',
               'DivDASqKM'], 
              axis=1,
              inplace=True)
plusflow[['FROMCOMID', 'TOCOMID']] = plusflow[['FROMCOMID', 'TOCOMID']].astype(str)
print(plusflow.dtypes)

FROMCOMID    object
TOCOMID      object
dtype: object


In [4]:
flowline = gpd.read_file(flowline_file)
flowline.drop(['FLOWDIR',
               'WBAREACOMI',
               'ENABLED',
               'GNIS_NBR'],
              axis=1,
              inplace=True)
flowline[['COMID', 'REACHCODE']] = flowline[['COMID', 'REACHCODE']].astype(str)
print(flowline.dtypes)

COMID           object
FDATE           object
RESOLUTION      object
GNIS_ID         object
GNIS_NAME       object
LENGTHKM       float64
REACHCODE       object
FTYPE           object
FCODE            int64
SHAPE_LENG     float64
geometry      geometry
dtype: object


In [5]:
# flowline.explore()

## Initialize KG

In [6]:
GCX = Namespace(f'https://geoconnex.us/')
GCX_CID = Namespace(f'https://geoconnex.us/nhdplusv2/comid/')
HYF = Namespace(f'https://www.opengis.net/def/schema/hy_features/hyf/')
SCHEMA = Namespace(f'https://schema.org/')
WDP = Namespace(f'https://www.wikidata.org/wiki/Property:')

In [7]:
kg = Graph(bind_namespaces="rdflib")
kg.bind('gcx', GCX)
kg.bind('gcx_cid', GCX_CID)
kg.bind('hyf', HYF)
kg.bind('schema', SCHEMA)
kg.bind('wdp', WDP)

## Functions

In [8]:
def getFlowDict(df) -> dict:
    dct = {}
    for row in df.itertuples():
        if row.FROMCOMID in dct.keys():
            dct[row.FROMCOMID].append(row.TOCOMID)
        else:
            dct[row.FROMCOMID] = [row.TOCOMID]
    return dct

## Process Data and Populate Graph

In [9]:
flow_dict = getFlowDict(plusflow)
grouping = str.maketrans('[]', '()')

for row in flowline.itertuples():
    comid = row.COMID
    reachIRI = GCX_CID[comid]
    reachGeoIRI = GCX_CID[comid + '.geometry']

    # There's an assumption that all reaches are drawn from head to outlet
    #    Based on a very small sample it is correct
    geom_json = json.loads(to_geojson(row.geometry))
    head = str(geom_json['coordinates'][0]).translate(grouping)
    outlet = str(geom_json['coordinates'][-1]).translate(grouping)
    
    # Create triples for current COMID
    #    Note: ReachCodes are not unique in NHDFlowline
    # This is based on Geoconnex as much as possible
    kg.add((reachIRI, RDF.type, SCHEMA['Place']))
    kg.add((reachIRI, RDF.type, HYF['HY_FlowPath']))
    kg.add((reachIRI, RDF.type, HYF['HY_Waterbody']))
    
    kg.add((reachIRI, GEO.hasGeometry, reachGeoIRI))
    kg.add((reachGeoIRI, GEO.asWKT, Literal(row.geometry, datatype=GEO.wktLiteral)))
    # Skipping schema:geo

    if not pd.isnull(row.GNIS_NAME):
        kg.add((reachIRI, SCHEMA['name'], Literal(row.GNIS_NAME, lang='en')))
    kg.add((reachIRI, RDFS.label, Literal(row.FTYPE, lang='en')))
    kg.add((reachIRI, RDFS.comment, Literal('USGS ReachCode: ' + row.REACHCODE)))
        
    # TODO: Add a unit (km) to P2043 via Q1978718
    kg.add((reachIRI, WDP['P2043'], Literal(row.LENGTHKM, datatype=XSD.float)))
    # Skipping P2053 since this data is not in NHDFlowline
    
    # In Geoconnex, P403 and P885 point to COMID objects (LineString objects), not nodes
    #    as well as HUC12 objects (Point objects). But not all point to HUC12 objects.
    # This creates new points:
    #    1: Create the point as a geo:Feature
    #    2: Assign the object a geo:Geometry
    #    3: Assign the geometry geo:asWKT coordinates
    #    4: Assign the feature to P403/P885 as the reach's outlet/head (see note above)
    outletIRI = GCX_CID[comid + '.outlet']
    outletGeoIRI = GCX_CID[comid + '.outlet.geometry']
    headIRI = GCX_CID[comid + '.head']
    headGeoIRI = GCX_CID[comid + '.head.geometry']
    
    kg.add((outletIRI, RDF.type, GEO.Feature))
    kg.add((outletIRI, GEO.hasGeometry, outletGeoIRI))
    kg.add((outletGeoIRI, GEO.asWKT, Literal('POINT ' + outlet, datatype=GEO.wktLiteral)))
    kg.add((reachIRI, WDP['P403'], outletIRI))
    
    kg.add((headIRI, RDF.type, GEO.Feature))
    kg.add((headIRI, GEO.hasGeometry, headGeoIRI))
    kg.add((headGeoIRI, GEO.asWKT, Literal('POINT ' + head, datatype=GEO.wktLiteral)))       
    kg.add((reachIRI, WDP['P885'], headIRI))
    
    if comid in flow_dict.keys():
        for cid in flow_dict[comid]:
            kg.add((reachIRI, HYF['downstreamWaterbody'], GCX_CID[cid]))
    kg.add((reachIRI, HYF['downstreamWaterbody'], reachIRI))
    # Skipping hyf:encompassingCatchment

## Create Turtle File

In [10]:
kg.serialize(ttl_file, format='turtle', base='http://geoconnex.us/ref/reaches/')

<Graph identifier=N7d073bd4759d49c88d46e2d1b8fe5eda (<class 'rdflib.graph.Graph'>)>